# Class 12 (25.10.2021)

# SQLite
SQLite is a software library that implements a self-contained, serverless, zero-configuration, transactional SQL database engine. It is the most widely deployed SQL database engine in the world. The source code for SQLite is in the public domain.
SQLite is not directly comparable to Client/Server SQL database engines such as MySQL, Oracle, PostgreSQL, or SQL Server since SQLite is trying to solve a different problem. 
Client/Server SQL database engines strive to implement a shared repository of enterprise data. They emphasize scalability, concurrency, centralization, and control. SQLite strives to provide local data storage for individual applications and devices. SQLite emphasizes economy, efficiency, reliability, independence, and simplicity. 

## When to avoid SQLite?
* When there are many client programs sending SQL requests to the same database over the network, a Client-Server databse would be better than SQLite.
* High volume websites (database has to be write intensive).<br/>
  SQLite will normally work fine as the database backend to a website. But if the website is write-intensive or is so busy that it requires multiple servers, then consider using an enterprise-class Client-Server database engine instead of SQLite.
* Very large scale databases. (SQLite is limited to 140TB {(247 bytes, 128 tibibytes)})
* High Concurrency<br/>
  SQLite supports an unlimited number of simultaneous readers, but it will only allow one writer at any instant in time.

## Connection objects
Connection objects create a connection with the database and these are further used for different transactions. These connection objects are also used as representatives of the database session.
You can use a connection object for calling methods like **commit()**, **rollback()** and **close()**

### Cursor objects
Cursor is one of the powerful features of SQL. These are objects that are responsible for submitting various SQL statements to a database server. 

<br/>
 
```python 
sqlite3.connect(database [,timeout ,other optional arguments])
```
This API opens a connection to the SQLite database file. You can use **":memory:"** to open a database connection to a database that resides in RAM instead of on disk. If database is opened successfully, it returns a connection object.
When a database is accessed by multiple connections, and one of the processes modifies the database, the SQLite database is locked until that transaction is committed. The timeout parameter specifies how long the connection should wait for the lock to go away until raising an exception. The default for the timeout parameter is 5.0 (five seconds).
If the given database name does not exist then this call will create the database. You can specify filename with the required path as well if you want to create a database anywhere else (other than the current directory).


<br/>

```python
connection.cursor([cursorClass])
```
This routine creates a cursor which will be used throughout our database programming with Python. This method accepts a single optional parameter **cursorClass**. If supplied, this must be a custom cursor class that extends **sqlite3.Cursor**.


<br/>

```python 
cursor.execute(sql [, optional parameters])
```
This routine executes an SQL statement. The SQL statement may be parameterized (i.e, placeholders instead of SQL literals). The sqlite3 module supports two kinds of placeholders: question marks and named placeholders (named style).
For example − **cursor.execute("insert into people values (?, ?)", (who, age))**


<br/>

```python 
connection.commit()
```
This method commits the current transaction. If you don't call this method, anything you did since the last call to commit() is not visible from other database connections.


<br/>

```python 
connection.rollback()
```
This method rolls back any changes to the database since the last call to commit


<br/>

```python
connection.close()
```
This method closes the database connection. Note that this does not automatically call commit(). If you just close your database connection without calling commit() first, your changes will be lost!

In [1]:
import sqlite3

# Connecting to Database
connection = sqlite3.connect("College.db") # If database is not existing, it will be created and stored on hard disk
cursor = connection.cursor() # Cursor object is used to perform actions on the database (SQL statements)

In [2]:
# Creating tables
#connection.execute("create table tab1(name char(20), age integer)") # This also works
cursor.execute("create table tab1(name char(20), age integer)")
cursor.execute("create table tab2(name char(20), age integer)")

In [3]:
# Inserting values
cursor.execute("insert into tab1 values('xyz', 21)")
cursor.execute("insert into tab2 values('abc', 12)")

# Using placeholders
info = ('efg', 54)
cursor.execute("insert into tab1(name, age) values(?, ?)", info)

In [4]:
# Displaying the table
tab1 = cursor.execute("select * from tab1")
for i in tab1: print(i)

print("-------------------")

tab2 = cursor.execute("select * from tab2")
for i in tab2: print(i)

('xyz', 21)
('efg', 54)
-------------------
('abc', 12)


In [5]:
connection.commit()
connection.close() # Closing the connection to the database

## Creating the Database in RAM

In [6]:
import sqlite3

# Each time a DB is created in RAM, it occupies a new region of memory. Hence one can create multiple independent DBs in RAM
connection =  sqlite3.connect(":memory:") # DB is created in RAM instead of HDD
cursor = connection.cursor()
cursor.execute("create table people(name, age)")

# QMark Style of insertion
who = 'Abc'
age = 21
cursor.execute("insert into people values(?,?)", (who, age))

# Named Style of insertion
cursor.execute("select * from people where name = :who and age = :age", {"who":who, "age":age})
print(cursor.fetchone())

connection.close()

('Abc', 21)


## Commit

In [7]:
import sqlite3
connection = sqlite3.connect("Company.db")

In [8]:
connection.execute('''
    create table Company (
        ID int primary key not null,
        Name text not null,
        Age int not null,
        Address char(50),
        Salary real
    );
''')

In [9]:
connection.execute('''
    insert into Company (ID, Name, Age, Address, Salary) values
        (1, 'ABC', 21, 'Bengaluru 560050', 1200000),
        (2, 'DEF', 22, 'Bengaluru 560040', 1100000),
        (3, 'GHI', 20, 'Bengaluru 560051', 1250000);
''')

connection.commit()
print(f"{connection.total_changes} Records inserted successfully")

3 Records inserted successfully


In [10]:
items = connection.execute("select * from Company")
columns = ("ID", "Name", "Age", "Address", "Salary")
for item in items: print(dict(zip(columns, item)))

{'ID': 1, 'Name': 'ABC', 'Age': 21, 'Address': 'Bengaluru 560050', 'Salary': 1200000.0}
{'ID': 2, 'Name': 'DEF', 'Age': 22, 'Address': 'Bengaluru 560040', 'Salary': 1100000.0}
{'ID': 3, 'Name': 'GHI', 'Age': 20, 'Address': 'Bengaluru 560051', 'Salary': 1250000.0}


In [11]:
connection.execute("update Company set Salary = 3000000 where ID = 3")

items = connection.execute("select * from Company")
columns = ("ID", "Name", "Age", "Address", "Salary")
for item in items: print(dict(zip(columns, item)))

{'ID': 1, 'Name': 'ABC', 'Age': 21, 'Address': 'Bengaluru 560050', 'Salary': 1200000.0}
{'ID': 2, 'Name': 'DEF', 'Age': 22, 'Address': 'Bengaluru 560040', 'Salary': 1100000.0}
{'ID': 3, 'Name': 'GHI', 'Age': 20, 'Address': 'Bengaluru 560051', 'Salary': 3000000.0}


In [12]:
connection.close()

# Class 13 (8.11.2021)

# Fetching all rows of a query result
* **cursor.fetchall()** <br/>
  Fetches all rows of a query result.<br/>
  It returns all rows as list of tuples.<br/>
  An empty list will be returned if there are no records to fetch.<br/><br/>

* **cursor.fetchmany(size)** <br/>
  Returns the number of rows specified by the size argument.<br/>
  It fetches the next set of rows when called repeatedly.<br/>
  Default value for size is 1.<br/>
  This also returns a list of tuples (an empty list will be returned if there are no records to fetch).<br/><br/>

* **cursor.fetchone()** <br/>
  Returns a single row that matches the condition. (None is returned if there are no rows to fetch).


In [13]:
import sqlite3
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()

In [14]:
# Only one statement can be executed at a time. 
# Hence we need to write two connection.execute() statements or use connection.executemany()
connection.execute('''
    create table supplier_groups(
        group_id int primary key,
        group_name text not null
    );
''')

connection.execute('''
    create table suppliers(
        supplier_id int primary key,
        supplier_name text not null,
        group_id int not null,
        foreign key(group_id) references supplier_groups(group_id)
    );
''')

In [15]:
connection.execute('''
    insert into supplier_groups values (1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg');
''')

In [16]:
ret = connection.execute("select * from supplier_groups")
for group in ret:
    print(group)

(1, 'abc')
(2, 'abc')
(3, 'xyz')
(4, 'efg')


In [17]:
cursor.execute("select * from supplier_groups")
p = cursor.fetchmany() # Default size is one
print(p)

[(1, 'abc')]


In [18]:
cursor.execute("select * from supplier_groups")
p1 = cursor.fetchmany(2)
p2 = cursor.fetchmany(2)
print(p1)
print(p2)

[(1, 'abc'), (2, 'abc')]
[(3, 'xyz'), (4, 'efg')]


In [19]:
cursor.execute("select * from supplier_groups")
all = cursor.fetchall()
print(all)

[(1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg')]


In [20]:
cursor.execute("select * from supplier_groups")
one = cursor.fetchone()
print(one)

(1, 'abc')


In [21]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchall())

[(1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg')]


In [22]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchone())

(1, 'abc')


In [23]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchmany(2))
print(ret.fetchmany(2))

[(1, 'abc'), (2, 'abc')]
[(3, 'xyz'), (4, 'efg')]


In [24]:
connection.close()

# SQLAlchemy (A powerful Open Source, Cross Platform SQL toolkit written in Python)
* This is a powerful database access toolkit for Python.
* It has Object Relational Mapper (ORM).
* ORM sits between SQLite database and Python program and transforms the dataflow between the database engine and Python object.
* SQLAlchemy allows us to think in terms of objects, i.e, provides an OOPS abstraction to SQL databases.
+ ORM is a programming technique for converting the data between incompatible type systems in Object Oriented Programming Languages.
+ The type system used in OOP languages like Python is Non Scalar, i.e, they cannot be expressed in terms of primitive datatypes like int, float, char, etc.
+ OOP languages therefore have to convert Non Scalar data into Scalar in order to interact with the backend data.
+ ORM maps a class to a table. This saves us from writing tedious database interface codes.

In [25]:
!pip install sqlalchemy

In [26]:
import sqlalchemy
print(sqlalchemy.__version__)

1.4.27


# Engine Class
* Connects a *Pool* and *Dialect* together. 
* This provides a source for database connectivity and behaviour.
* An object of Engine class is created using the *create_engine* function.

In [27]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, ForeignKey

# SQL Expression language constructs its expressions against table columns
engine = create_engine('sqlite:///College.db', echo = True) # echo makes the output (SQL statements) verbose

# MetaData is a collection of table objects and their associated schema constructs.
meta = MetaData() # Contains info about the tables and associated objects like index, view, triggers, etc.

# Creating tables
sections = Table(
    'Sections', meta,
    Column('ID', Integer, primary_key = True),
    Column('Floor', Integer)
)

students = Table(
    'Students', meta,
    Column('ID', Integer, primary_key = True),
    Column('Name', String),
    Column('LastName', String),
    Column('Section', Integer, ForeignKey('Sections.ID')),
)

meta.create_all(engine) # Creates the table
connection = engine.connect()

2021-11-12 14:40:26,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-12 14:40:26,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sections")
2021-11-12 14:40:26,582 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-12 14:40:26,589 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sections")
2021-11-12 14:40:26,592 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-12 14:40:26,597 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Students")
2021-11-12 14:40:26,600 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-12 14:40:26,604 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Students")
2021-11-12 14:40:26,606 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-12 14:40:26,614 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sections" (
	"ID" INTEGER NOT NULL, 
	"Floor" INTEGER, 
	PRIMARY KEY ("ID")
)


2021-11-12 14:40:26,616 INFO sqlalchemy.engine.Engine [no key 0.00215s] ()
2021-11-12 14:40:26,781 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Students" (


# Class 14 (10.11.2021)

In [28]:
connection.execute(sections.insert(), [
    {'ID': 1, 'Floor' : 1},
    {'ID': 2, 'Floor' : 0}
])

connection.execute(students.insert(), [
    {'ID': 1, 'Name' : 'Abc', 'LastName' : 'Efg', 'Section': 2},
    {'ID': 2, 'Name' : 'Def', 'LastName' : 'Ghi', 'Section': 1}
])

2021-11-12 14:40:27,012 INFO sqlalchemy.engine.Engine INSERT INTO "Sections" ("ID", "Floor") VALUES (?, ?)
2021-11-12 14:40:27,014 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ((1, 1), (2, 0))
2021-11-12 14:40:27,016 INFO sqlalchemy.engine.Engine COMMIT
2021-11-12 14:40:27,150 INFO sqlalchemy.engine.Engine INSERT INTO "Students" ("ID", "Name", "LastName", "Section") VALUES (?, ?, ?, ?)
2021-11-12 14:40:27,151 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ((1, 'Abc', 'Efg', 2), (2, 'Def', 'Ghi', 1))
2021-11-12 14:40:27,152 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
result = connection.execute(students.select())
print(f'\nresult = {result}')
for row in result: print(row)

2021-11-12 14:40:27,267 INFO sqlalchemy.engine.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName", "Students"."Section" 
FROM "Students"
2021-11-12 14:40:27,269 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ()

result = <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f42a439ac40>
(1, 'Abc', 'Efg', 2)
(2, 'Def', 'Ghi', 1)


In [30]:
# Select with condition
filtered = connection.execute(students.select().where(students.c.ID > 1))
for row in filtered: print(row)

2021-11-12 14:40:27,822 INFO sqlalchemy.engine.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName", "Students"."Section" 
FROM "Students" 
WHERE "Students"."ID" > ?
2021-11-12 14:40:27,824 INFO sqlalchemy.engine.Engine [generated in 0.00187s] (1,)
(2, 'Def', 'Ghi', 1)


In [31]:
# Update
connection.execute(students.update().where(students.c.Name == 'Abc').values(Name = 'Xyz'))
result = connection.execute(students.select())
print(f'result = {result}')
for row in result: print(row)

2021-11-12 14:40:28,035 INFO sqlalchemy.engine.Engine UPDATE "Students" SET "Name"=? WHERE "Students"."Name" = ?
2021-11-12 14:40:28,036 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ('Xyz', 'Abc')
2021-11-12 14:40:28,038 INFO sqlalchemy.engine.Engine COMMIT
2021-11-12 14:40:28,218 INFO sqlalchemy.engine.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName", "Students"."Section" 
FROM "Students"
2021-11-12 14:40:28,219 INFO sqlalchemy.engine.Engine [cached since 0.9522s ago] ()
result = <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f429404e670>
(1, 'Xyz', 'Efg', 2)
(2, 'Def', 'Ghi', 1)


In [32]:
# Delete
connection.execute(students.delete().where(students.c.LastName == 'Ghi'))
result = connection.execute(students.select())
print(f'result = {result}')
for row in result: print(row)

2021-11-12 14:40:28,258 INFO sqlalchemy.engine.Engine DELETE FROM "Students" WHERE "Students"."LastName" = ?
2021-11-12 14:40:28,262 INFO sqlalchemy.engine.Engine [generated in 0.00394s] ('Ghi',)
2021-11-12 14:40:28,267 INFO sqlalchemy.engine.Engine COMMIT
2021-11-12 14:40:28,450 INFO sqlalchemy.engine.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName", "Students"."Section" 
FROM "Students"
2021-11-12 14:40:28,451 INFO sqlalchemy.engine.Engine [cached since 1.184s ago] ()
result = <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f42a4363940>
(1, 'Xyz', 'Efg', 2)


In [33]:
connection.close()

## Connecting to an existing database in SQLAlchemy

In [34]:
engine = create_engine('sqlite:///College.db', echo = True)
print(engine)

Engine(sqlite:///College.db)


In [37]:
connection = engine.connect()
print(connection)

In [40]:
res = connection.execute(students.select())
print(res.fetchall()) # Similarly one can use fethone and fetchmany as well

2021-11-12 14:43:24,768 INFO sqlalchemy.engine.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName", "Students"."Section" 
FROM "Students"
2021-11-12 14:43:24,770 INFO sqlalchemy.engine.Engine [cached since 24.77s ago] ()
[(1, 'Xyz', 'Efg', 2)]


In [41]:
connection.close()